# Preparação e Estruturação dos Dados Brutos (Camada RAW)

Este notebook é responsável por ler os dados coletados na camada Landing (LND), aplicando esquemas estruturados para transformar os arquivos JSON e CSV em DataFrames do PySpark. Os dados são então organizados, renomeados e armazenados no formato Delta Lake na camada RAW, com particionamento para otimização do armazenamento e consultas futuras.

## Configuração e Importações do pyspark e delta lake

In [37]:
# Para iniciar a seção spark
from typing import Optional, Union, List
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql import functions as f
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable

In [38]:
builder: SparkSession.Builder = SparkSession.builder \
    .appName("App Preparação RAW") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g")

spark: SparkSession = configure_spark_with_delta_pip(builder).getOrCreate()

spark

25/07/24 10:03:54 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


## Funções para evitar repetição

In [3]:
# Apenas para arquivos com formato CSV.
def ler_csv(
        path: str,
        encoding: str,
        schema: Optional[StructType] = None,
        header: bool = False,
        sep: str = ";",
        quote: str = '"'
) -> DataFrame:
    """
    Lê um arquivo CSV usando PySpark com esquema definido e opções customizáveis.

    Esta função lê um arquivo CSV a partir do caminho especificado, aplicando o esquema
    fornecido para garantir tipos corretos nas colunas. Permite configurar o encoding,
    se o arquivo possui cabeçalho, o separador de campo e o caractere de aspas.

    Args:
        path (str): Caminho do arquivo CSV a ser lido.
        schema (StructType): Esquema (schema) PySpark para aplicar ao DataFrame.
        encoding (str): Codificação do arquivo CSV, ex: "UTF-8", "ISO-8859-1", etc.
        header (bool, optional): Indica se o arquivo possui linha de cabeçalho com nomes das colunas.
            Padrão é False.
        sep (str, optional): Caractere separador dos campos no CSV. Padrão é ponto e vírgula ";".
        quote (str, optional): Caractere usado para delimitar campos com texto. Padrão é aspas duplas '"'.

    Returns:
        DataFrame: DataFrame do Spark contendo os dados lidos do CSV conforme o esquema informado.

    """
    try:
        # Inicializa a leitura do arquivo CSV com o formato adequado
        reader = spark.read.format("csv") \
            .option("header", header) \
            .option("sep", sep) \
            .option("quote", quote) \
            .option("encoding", encoding)

        # Se foi passado schema explícito, aplica o schema fixo para o DataFrame
        if schema:
            reader = reader.schema(schema)
        else:
            # Se não, ativa inferência automática de esquema pelo Spark
            reader = reader.option("inferSchema", True)

        # Carrega o arquivo CSV do caminho informado com as opções configuradas
        df = reader.load(path)

        # Retorna o DataFrame carregado
        return df

    # Captura qualquer erro na leitura, imprime a mensagem e pode ser melhorado para levantar erro
    except Exception as e:
        print(f"[ERRO] Falha na leitura do CSV: {e}")
        raise

In [4]:
def gravar_delta(
    df: DataFrame,
    path: str,
    mode: str = "overwrite",
    partitionBy: Optional[Union[str, List[str]]] = None
) -> DataFrame:
    """
    Grava um DataFrame no formato Delta Lake em um caminho especificado.

    Salva o DataFrame no formato Delta, permitindo definir o modo de gravação e colunas
    para particionamento dos dados. Útil para organizar dados grandes em diretórios particionados.

    Args:
        df (DataFrame): DataFrame do Spark que será salvo.
        path (str): Caminho do diretório onde os dados serão gravados.
        mode (str, optional): Modo de gravação, como 'overwrite' (sobrescrever), 'append' (acrescentar),
            entre outros suportados pelo Spark. Default é 'overwrite'.
        partitionBy (str ou List[str], optional): Coluna ou lista de colunas para particionamento dos dados.
            Se None, não realiza particionamento. Default é None.

    Returns:
        DataFrame: Retorna o mesmo DataFrame recebido como argumento para facilitar encadeamento.
    """
    try:
        # Cria um escritor de DataFrame para o formato Delta
        writer = df.write.format("delta").mode(mode)

        # Se foram informadas colunas para particionamento, ajusta o escritor para particionar
        if partitionBy:

            # Se for uma string só, transforma em lista para unpack
            if isinstance(partitionBy, str):
                partitionBy = [partitionBy]

            # Aplica o particionamento passando todas as colunas indicadas
            writer = writer.partitionBy(*partitionBy)

        # Salva o DataFrame no caminho especificado no formato Delta
        writer.save(path)

        # Mensagem de sucesso na gravação
        print("Gravação efetuada!")

        # Retorna o DataFrame original para permitir encadeamento de operações, se desejar
        return df

    except Exception as e:
        # Captura qualquer erro durante a gravação, imprime mensagem de erro
        print(f"[ERRO] Falha ao gravar Delta no caminho '{path}': {e}")
        raise

## RAW - Dados IBGE

In [22]:
# Caminho do arquivo

path_json_ibge_municipios: str = "../LND/municipios/municipios_2025-07-23_200927.json"

In [23]:
# Definindo schema para o JSON aninhado dos municípios IBGE
schema_ibge_municipios: StructType = StructType([
    StructField("id", StringType(), True),
    StructField("nome", StringType(), True),
    StructField("microrregiao", StructType([
        StructField("id", StringType(), True),
        StructField("nome", StringType(), True),
        StructField("mesorregiao", StructType([
            StructField("id", StringType(), True),
            StructField("nome", StringType(), True),
            StructField("UF", StructType([
                StructField("id", StringType(), True),
                StructField("sigla", StringType(), True),
                StructField("nome", StringType(), True),
                StructField("regiao", StructType([
                    StructField("id", StringType(), True),
                    StructField("sigla", StringType(), True),
                    StructField("nome", StringType(), True),
                ]))
            ]))
        ]))
    ])),
    StructField("regiao-imediata", StructType([
        StructField("id", StringType(), True),
        StructField("nome", StringType(), True),
        StructField("regiao-intermediaria", StructType([
            StructField("id", StringType(), True),
            StructField("nome", StringType(), True),
            StructField("UF", StructType([
                StructField("id", StringType(), True),
                StructField("sigla", StringType(), True),
                StructField("nome", StringType(), True),
                StructField("regiao", StructType([
                    StructField("id", StringType(), True),
                    StructField("sigla", StringType(), True),
                    StructField("nome", StringType(), True),
                ]))
            ]))
        ]))
    ]))
])

In [24]:
try:
    # Lê o arquivo JSON com múltiplas linhas, aplicando o schema previamente definido
    df_ibge_municipios: DataFrame = spark.read \
        .option("multiline", True) \
        .schema(schema_ibge_municipios) \
        .json(path_json_ibge_municipios)

    # Caso ocorra qualquer erro durante a leitura, imprime a mensagem e relança o erro para controle externo
except Exception as e:
    print(f"Erro: {e}")
    raise

In [25]:
# Dados após seleção
df_ibge_municipios.show(5, truncate=False)

+-------+---------------------+----------------------------------------------------------------------------------------+---------------------------------------------------------------------------+
|id     |nome                 |microrregiao                                                                            |regiao-imediata                                                            |
+-------+---------------------+----------------------------------------------------------------------------------------+---------------------------------------------------------------------------+
|1100015|Alta Floresta D'Oeste|{11006, Cacoal, {1102, Leste Rondoniense, {11, RO, Rondônia, {1, N, Norte}}}}           |{110005, Cacoal, {1102, Ji-Paraná, {11, RO, Rondônia, {1, N, Norte}}}}     |
|1100023|Ariquemes            |{11003, Ariquemes, {1102, Leste Rondoniense, {11, RO, Rondônia, {1, N, Norte}}}}        |{110002, Ariquemes, {1101, Porto Velho, {11, RO, Rondônia, {1, N, Norte}}}}|
|1100031|Cabixi

In [29]:
df_ibge_municipios = df_ibge_municipios.withColumn(
    "data_ingestao",
    f.current_timestamp()
)

df_ibge_municipios.show(truncate=False)

+-------+------------------------+----------------------------------------------------------------------------------------+-----------------------------------------------------------------------------+--------------------------+
|id     |nome                    |microrregiao                                                                            |regiao-imediata                                                              |data_ingestao             |
+-------+------------------------+----------------------------------------------------------------------------------------+-----------------------------------------------------------------------------+--------------------------+
|1100015|Alta Floresta D'Oeste   |{11006, Cacoal, {1102, Leste Rondoniense, {11, RO, Rondônia, {1, N, Norte}}}}           |{110005, Cacoal, {1102, Ji-Paraná, {11, RO, Rondônia, {1, N, Norte}}}}       |2025-07-23 23:10:38.421054|
|1100023|Ariquemes               |{11003, Ariquemes, {1102, Leste Rondoniense, {11, 

In [ ]:
# Escrever dados com Delta Lake no formato parquet.

gravar_delta(df_ibge_municipios, "../RAW/municipios_ibge")

25/07/23 23:12:21 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Gravação efetuada!


DataFrame[id: string, nome: string, microrregiao: struct<id:string,nome:string,mesorregiao:struct<id:string,nome:string,UF:struct<id:string,sigla:string,nome:string,regiao:struct<id:string,sigla:string,nome:string>>>>, regiao-imediata: struct<id:string,nome:string,regiao-intermediaria:struct<id:string,nome:string,UF:struct<id:string,sigla:string,nome:string,regiao:struct<id:string,sigla:string,nome:string>>>>, data_ingestao: timestamp]

In [5]:
try:
    # Tenta criar um objeto DeltaTable apontando para o caminho Delta na camada RAW
    deltaTable_municipios_ibge_raw: DeltaTable = DeltaTable.forPath(spark, "../RAW/municipios_ibge")

    # Converte o DeltaTable para DataFrame para manipulação e análise
    df_municipios_ibge_raw: DataFrame = deltaTable_municipios_ibge_raw.toDF()

    # Exibe os dados carregados, sem truncar as colunas para melhor visualização
    # Além do schema
    df_municipios_ibge_raw.printSchema()
    df_municipios_ibge_raw.show(truncate=False)

# Captura e exibe qualquer erro ocorrido durante a leitura do DeltaTable
except Exception as e:
    print(f"[ERRO] Falha na leitura do DeltaTable: {e}")
    raise

root
 |-- id: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- microrregiao: struct (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- nome: string (nullable = true)
 |    |-- mesorregiao: struct (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- nome: string (nullable = true)
 |    |    |-- UF: struct (nullable = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- sigla: string (nullable = true)
 |    |    |    |-- nome: string (nullable = true)
 |    |    |    |-- regiao: struct (nullable = true)
 |    |    |    |    |-- id: string (nullable = true)
 |    |    |    |    |-- sigla: string (nullable = true)
 |    |    |    |    |-- nome: string (nullable = true)
 |-- regiao-imediata: struct (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- nome: string (nullable = true)
 |    |-- regiao-intermediaria: struct (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- nome

25/07/24 09:48:41 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------------+----------------------------------------------------------------------------------------+-----------------------------------------------------------------------------+--------------------------+
|id     |nome                    |microrregiao                                                                            |regiao-imediata                                                              |data_ingestao             |
+-------+------------------------+----------------------------------------------------------------------------------------+-----------------------------------------------------------------------------+--------------------------+
|1100015|Alta Floresta D'Oeste   |{11006, Cacoal, {1102, Leste Rondoniense, {11, RO, Rondônia, {1, N, Norte}}}}           |{110005, Cacoal, {1102, Ji-Paraná, {11, RO, Rondônia, {1, N, Norte}}}}       |2025-07-23 23:12:11.297894|
|1100023|Ariquemes               |{11003, Ariquemes, {1102, Leste Rondoniense, {11, 

## RAW - Dados CNAE

In [6]:
# Caminho para o CSV na camada LND
path_csv_cnae: str = "../LND/cnaes/2025-06/Cnaes_2025-07-23_205727.csv"

In [7]:
# Schema esperado para o arquivo CNAE
schema_cnae: StructType = StructType([
    StructField("codigo_cnae", StringType(), True),
    StructField("descricao_cnae", StringType(), True)
])

In [8]:
# Leitura do arquivo CSV com o schema definido
df_cnae = ler_csv(path_csv_cnae, "ISO-8859-1", schema_cnae)

# Visualizar os dados carregados
df_cnae.show(truncate=False)

+-----------+-----------------------------------------------------------------------------------+
|codigo_cnae|descricao_cnae                                                                     |
+-----------+-----------------------------------------------------------------------------------+
|0111301    |Cultivo de arroz                                                                   |
|0111302    |Cultivo de milho                                                                   |
|0111303    |Cultivo de trigo                                                                   |
|0111399    |Cultivo de outros cereais não especificados anteriormente                          |
|0112101    |Cultivo de algodão herbáceo                                                        |
|0112102    |Cultivo de juta                                                                    |
|0112199    |Cultivo de outras fibras de lavoura temporária não especificadas anteriormente     |
|0113000    |Cultivo

In [9]:
df_cnae = df_cnae.withColumn(
    "data_ingestao",
    f.current_timestamp()
)

df_cnae.show(truncate=False)

+-----------+-----------------------------------------------------------------------------------+--------------------------+
|codigo_cnae|descricao_cnae                                                                     |data_ingestao             |
+-----------+-----------------------------------------------------------------------------------+--------------------------+
|0111301    |Cultivo de arroz                                                                   |2025-07-24 09:49:42.714784|
|0111302    |Cultivo de milho                                                                   |2025-07-24 09:49:42.714784|
|0111303    |Cultivo de trigo                                                                   |2025-07-24 09:49:42.714784|
|0111399    |Cultivo de outros cereais não especificados anteriormente                          |2025-07-24 09:49:42.714784|
|0112101    |Cultivo de algodão herbáceo                                                        |2025-07-24 09:49:42.714784|


In [10]:
# Escrita no formato Delta na camada RAW

gravar_delta(df_cnae, "../RAW/cnae")

Gravação efetuada!


DataFrame[codigo_cnae: string, descricao_cnae: string, data_ingestao: timestamp]

In [11]:
# Leitura da Delta Table salva para validação
try:
    # Tenta criar um objeto DeltaTable apontando para o caminho Delta na camada RAW
    deltaTable_cnae_raw: DeltaTable = DeltaTable.forPath(spark, "../RAW/cnae")

    # Converte o DeltaTable para DataFrame para manipulação e análise
    df_cnae_raw: DataFrame = deltaTable_cnae_raw.toDF()

    # Exibe os dados carregados, sem truncar as colunas para melhor visualização
    # Além do schema
    df_cnae_raw.printSchema()
    df_cnae_raw.show(truncate=False)

# Captura e exibe qualquer erro ocorrido durante a leitura do DeltaTable
except Exception as e:
    print(f"[ERRO] Falha na leitura do DeltaTable: {e}")
    raise

root
 |-- codigo_cnae: string (nullable = true)
 |-- descricao_cnae: string (nullable = true)
 |-- data_ingestao: timestamp (nullable = true)

+-----------+-----------------------------------------------------------------------------------+--------------------------+
|codigo_cnae|descricao_cnae                                                                     |data_ingestao             |
+-----------+-----------------------------------------------------------------------------------+--------------------------+
|0111301    |Cultivo de arroz                                                                   |2025-07-24 09:49:47.363057|
|0111302    |Cultivo de milho                                                                   |2025-07-24 09:49:47.363057|
|0111303    |Cultivo de trigo                                                                   |2025-07-24 09:49:47.363057|
|0111399    |Cultivo de outros cereais não especificados anteriormente                          |2025-07-24

## RAW - Natureza Jurídica

In [16]:
# Caminho para o arquivo CSV da natureza jurídica
path_csv_natureza_juridica: str = "../LND/naturezas/2025-06/Naturezas_2025-07-23_211057.csv"

In [17]:
# Definição do schema para o CSV de natureza jurídica
schema_natureza_juridica: StructType = StructType([
    StructField("codigo_natureza_juridica", StringType(), True),
    StructField("natureza_juridica", StringType(), True),
])

In [ ]:
# Leitura do arquivo CSV com o schema definido
df_natureza_juridica: DataFrame = ler_csv(path_csv_natureza_juridica, "iso-8859-1", schema_natureza_juridica)

In [20]:
# Visualização inicial dos dados
df_natureza_juridica.show(truncate=False)

+------------------------+------------------------------------------------------------------+
|codigo_natureza_juridica|natureza_juridica                                                 |
+------------------------+------------------------------------------------------------------+
|0000                    |Natureza Jurídica não informada                                   |
|3271                    |Órgão de Direção Local de Partido Político                        |
|3280                    |Comitê Financeiro de Partido Político                             |
|3298                    |Frente Plebiscitária ou Referendária                              |
|3301                    |Organização Social (OS)                                           |
|3999                    |Associação Privada                                                |
|4014                    |Empresa Individual Imobiliária                                    |
|4090                    |Candidato a Cargo Político Eletivo

In [21]:
df_natureza_juridica = df_natureza_juridica.withColumn(
    "data_ingestao",
    f.current_timestamp()
)

df_natureza_juridica.show(truncate=False)

+------------------------+------------------------------------------------------------------+--------------------------+
|codigo_natureza_juridica|natureza_juridica                                                 |data_ingestao             |
+------------------------+------------------------------------------------------------------+--------------------------+
|0000                    |Natureza Jurídica não informada                                   |2025-07-24 09:52:53.578933|
|3271                    |Órgão de Direção Local de Partido Político                        |2025-07-24 09:52:53.578933|
|3280                    |Comitê Financeiro de Partido Político                             |2025-07-24 09:52:53.578933|
|3298                    |Frente Plebiscitária ou Referendária                              |2025-07-24 09:52:53.578933|
|3301                    |Organização Social (OS)                                           |2025-07-24 09:52:53.578933|
|3999                    |Associ

In [22]:
# Escrita dos dados no formato Delta, particionando por código
try:
    gravar_delta(df_natureza_juridica, "../RAW/natureza_juridica")
except Exception as e:
    print(f"[ERRO] Falha na gravação do DeltaTable")
    raise

Gravação efetuada!


In [23]:
# Leitura da tabela Delta para validação
try:
    deltaTable_nj = DeltaTable.forPath(spark, "../RAW/natureza_juridica")
    df_natureza_juridica: DataFrame = deltaTable_nj.toDF()
    df_natureza_juridica.printSchema()
    df_natureza_juridica.show(truncate=False)
except Exception as e:
    print(f"[ERRO] Falha na leitura do DeltaTable: {e}")
    raise

root
 |-- codigo_natureza_juridica: string (nullable = true)
 |-- natureza_juridica: string (nullable = true)
 |-- data_ingestao: timestamp (nullable = true)

+------------------------+------------------------------------------------------------------+--------------------------+
|codigo_natureza_juridica|natureza_juridica                                                 |data_ingestao             |
+------------------------+------------------------------------------------------------------+--------------------------+
|0000                    |Natureza Jurídica não informada                                   |2025-07-24 09:53:45.817935|
|3271                    |Órgão de Direção Local de Partido Político                        |2025-07-24 09:53:45.817935|
|3280                    |Comitê Financeiro de Partido Político                             |2025-07-24 09:53:45.817935|
|3298                    |Frente Plebiscitária ou Referendária                              |2025-07-24 09:53:45.81

## RAW - Empresas

In [24]:
# Caminho do arquivo CSV
path_csv_empresas: str = "../LND/empresas/2025-06/Empresas1_2025-07-23_205727.csv"

In [25]:
# Definição do schema para o CSV de empresas
schema_empresas: StructType = StructType([
    StructField("cnpj_basico", StringType(), True),
    StructField("razao_social", StringType(), True),
    StructField("codigo_natureza_juridica", StringType(), True),
    StructField("codigo_qualificacao_responsavel", StringType(), True),
    StructField("capital_social", StringType(), True),
    StructField("codigo_porte_empresa", StringType(), True),
    StructField("ente_federativo_responsavel", StringType(), True)
])

In [26]:
# Leitura do arquivo CSV
df_empresas: DataFrame = ler_csv(path_csv_empresas, "iso-8859-1", schema_empresas)

In [27]:
df_empresas.show(5, truncate=False)

+-----------+------------------------------------------------------+------------------------+-------------------------------+---------------+--------------------+---------------------------+
|cnpj_basico|razao_social                                          |codigo_natureza_juridica|codigo_qualificacao_responsavel|capital_social |codigo_porte_empresa|ente_federativo_responsavel|
+-----------+------------------------------------------------------+------------------------+-------------------------------+---------------+--------------------+---------------------------+
|00000000   |BANCO DO BRASIL SA                                    |2038                    |10                             |120000000000,00|05                  |NULL                       |
|00000001   |ASSOCIACAO DE AMIGOS DE BAIRRO DO CONJ PAULISTANO     |3999                    |16                             |0,00           |05                  |NULL                       |
|00000002   |WM&R EMPREITEIRA DE CONSTRUCAO C

In [28]:
df_empresas = df_empresas.withColumn(
    "data_ingestao",
    f.current_timestamp()
)

df_empresas.show(truncate=False)

+-----------+------------------------------------------------------------+------------------------+-------------------------------+---------------+--------------------+---------------------------+--------------------------+
|cnpj_basico|razao_social                                                |codigo_natureza_juridica|codigo_qualificacao_responsavel|capital_social |codigo_porte_empresa|ente_federativo_responsavel|data_ingestao             |
+-----------+------------------------------------------------------------+------------------------+-------------------------------+---------------+--------------------+---------------------------+--------------------------+
|00000000   |BANCO DO BRASIL SA                                          |2038                    |10                             |120000000000,00|05                  |NULL                       |2025-07-24 09:59:25.653523|
|00000001   |ASSOCIACAO DE AMIGOS DE BAIRRO DO CONJ PAULISTANO           |3999                    |16   

In [29]:
#  Escrita no Delta Lake

gravar_delta(df_empresas, "../RAW/empresas")

25/07/24 09:59:28 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/07/24 09:59:28 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/07/24 09:59:28 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/07/24 09:59:28 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/07/24 09:59:28 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
25/07/24 09:59:34 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/07/24 09:59:34 WARN MemoryManager: Total allocation exceeds 95.

Gravação efetuada!


DataFrame[cnpj_basico: string, razao_social: string, codigo_natureza_juridica: string, codigo_qualificacao_responsavel: string, capital_social: string, codigo_porte_empresa: string, ente_federativo_responsavel: string, data_ingestao: timestamp]

In [39]:
try:
    # Leitura e verificação da DeltaTable
    deltaTable_empresas = DeltaTable.forPath(spark, "../RAW/empresas")
    df_empresas: DataFrame = deltaTable_empresas.toDF()
    df_empresas.printSchema()
    df_empresas.show(5, truncate=False)
except Exception as e:
    print(f"[ERRO] Falha na leitura da DeltaTable: {e}")

root
 |-- cnpj_basico: string (nullable = true)
 |-- razao_social: string (nullable = true)
 |-- codigo_natureza_juridica: string (nullable = true)
 |-- codigo_qualificacao_responsavel: string (nullable = true)
 |-- capital_social: string (nullable = true)
 |-- codigo_porte_empresa: string (nullable = true)
 |-- ente_federativo_responsavel: string (nullable = true)
 |-- data_ingestao: timestamp (nullable = true)

+-----------+------------------------------------------------------+------------------------+-------------------------------+---------------+--------------------+---------------------------+-------------------------+
|cnpj_basico|razao_social                                          |codigo_natureza_juridica|codigo_qualificacao_responsavel|capital_social |codigo_porte_empresa|ente_federativo_responsavel|data_ingestao            |
+-----------+------------------------------------------------------+------------------------+-------------------------------+---------------+--------

## RAW - Simples Nacional

In [31]:
# Caminho CSV
path_csv_simples_nacional: str = '../LND/simples/2025-06/Simples_2025-07-23_212004.csv'

In [32]:
# Schema do arquivo simples.csv
schema_simples_nacional: StructType = StructType([
    StructField("cnpj_basico", StringType(), True),
    StructField("opcao_simples", StringType(), True),
    StructField("data_opcao_simples", StringType(), True),
    StructField("data_exclusao_simples", StringType(), True),
    StructField("opcao_mei", StringType(), True),
    StructField("data_opcao_mei", StringType(), True),
    StructField("data_exclusao_mei", StringType(), True),
])

In [33]:
# # Leitura do CSV

df_simples_nacional = ler_csv(path_csv_simples_nacional, "iso-8859-1", schema_simples_nacional)

In [34]:
df_simples_nacional.show(5, truncate=False)

+-----------+-------------+------------------+---------------------+---------+--------------+-----------------+
|cnpj_basico|opcao_simples|data_opcao_simples|data_exclusao_simples|opcao_mei|data_opcao_mei|data_exclusao_mei|
+-----------+-------------+------------------+---------------------+---------+--------------+-----------------+
|00000000   |N            |20070701          |20070701             |N        |20090701      |20090701         |
|00000006   |N            |20180101          |20191231             |N        |00000000      |00000000         |
|00000008   |N            |20140101          |20211231             |N        |00000000      |00000000         |
|00000011   |S            |20070701          |00000000             |N        |00000000      |00000000         |
|00000013   |N            |20090101          |20231231             |N        |00000000      |00000000         |
+-----------+-------------+------------------+---------------------+---------+--------------+-----------

In [35]:
df_simples_nacional = df_simples_nacional.withColumn(
    "data_ingestao",
    f.current_timestamp()
)

df_simples_nacional.show(truncate=False)

+-----------+-------------+------------------+---------------------+---------+--------------+-----------------+--------------------------+
|cnpj_basico|opcao_simples|data_opcao_simples|data_exclusao_simples|opcao_mei|data_opcao_mei|data_exclusao_mei|data_ingestao             |
+-----------+-------------+------------------+---------------------+---------+--------------+-----------------+--------------------------+
|00000000   |N            |20070701          |20070701             |N        |20090701      |20090701         |2025-07-24 10:01:29.624701|
|00000006   |N            |20180101          |20191231             |N        |00000000      |00000000         |2025-07-24 10:01:29.624701|
|00000008   |N            |20140101          |20211231             |N        |00000000      |00000000         |2025-07-24 10:01:29.624701|
|00000011   |S            |20070701          |00000000             |N        |00000000      |00000000         |2025-07-24 10:01:29.624701|
|00000013   |N            |

In [36]:
# Escrita na camada RAW em Delta Lake

gravar_delta(df_simples_nacional, "../RAW/simples_nacional")

25/07/24 10:01:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/07/24 10:01:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/07/24 10:01:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/07/24 10:01:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/07/24 10:01:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
25/07/24 10:02:28 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/07/24 10:02:29 WARN MemoryManager: Total allocation exceeds 95.

Gravação efetuada!


DataFrame[cnpj_basico: string, opcao_simples: string, data_opcao_simples: string, data_exclusao_simples: string, opcao_mei: string, data_opcao_mei: string, data_exclusao_mei: string, data_ingestao: timestamp]

In [40]:
try:
    deltaTable_simples: DeltaTable = DeltaTable.forPath(spark, "../RAW/simples_nacional")
    df_simples_nacional: DataFrame = deltaTable_simples.toDF()
    df_simples_nacional.printSchema()
    df_simples_nacional.show(5, truncate=False)
except Exception as e:
    print(f"[ERRO] Falha na leitura da DeltaTable: {e}")

root
 |-- cnpj_basico: string (nullable = true)
 |-- opcao_simples: string (nullable = true)
 |-- data_opcao_simples: string (nullable = true)
 |-- data_exclusao_simples: string (nullable = true)
 |-- opcao_mei: string (nullable = true)
 |-- data_opcao_mei: string (nullable = true)
 |-- data_exclusao_mei: string (nullable = true)
 |-- data_ingestao: timestamp (nullable = true)

+-----------+-------------+------------------+---------------------+---------+--------------+-----------------+--------------------------+
|cnpj_basico|opcao_simples|data_opcao_simples|data_exclusao_simples|opcao_mei|data_opcao_mei|data_exclusao_mei|data_ingestao             |
+-----------+-------------+------------------+---------------------+---------+--------------+-----------------+--------------------------+
|47096018   |S            |20220711          |00000000             |S        |20220711      |00000000         |2025-07-24 10:01:48.825146|
|47096019   |S            |20220711          |00000000         

## RAW - Estabelecimentos

In [41]:
# Caminho do CSV
path_csv_estabelecimentos: str = '../LND/estabelecimentos/2025-06/Estabelecimentos1_2025-07-23_205820.csv'

In [42]:
# Definição do Schema
schema_estabelecimentos: StructType = StructType([
    StructField("cnpj_base", StringType(), True),
    StructField("cnpj_ordem", StringType(), True),
    StructField("cnpj_dv", StringType(), True),
    StructField("matriz_filial", StringType(), True),
    StructField("nome_fantasia", StringType(), True),
    StructField("situacao_cadastral", StringType(), True),
    StructField("data_situacao_cadastral", StringType(), True),
    StructField("motivo_situacao_cadastral", StringType(), True),
    StructField("nome_cidade_exterior", StringType(), True),
    StructField("pais", StringType(), True),
    StructField("data_inicio_atividade", StringType(), True),
    StructField("cnae_fiscal", StringType(), True),
    StructField("cnae_fiscal_secundaria", StringType(), True),
    StructField("tipo_logradouro", StringType(), True),
    StructField("logradouro", StringType(), True),
    StructField("numero", StringType(), True),
    StructField("complemento", StringType(), True),
    StructField("bairro", StringType(), True),
    StructField("cep", StringType(), True),
    StructField("uf", StringType(), True),
    StructField("codigo_municipio", StringType(), True),
    StructField("ddd1", StringType(), True),
    StructField("telefone1", StringType(), True),
    StructField("ddd2", StringType(), True),
    StructField("telefone2", StringType(), True),
    StructField("ddd_fax", StringType(), True),
    StructField("fax", StringType(), True),
    StructField("email", StringType(), True),
    StructField("situacao_especial", StringType(), True),
    StructField("data_situacao_especial", StringType(), True)
])

In [43]:
df_estabelecimentos: DataFrame = ler_csv(
    path_csv_estabelecimentos, 
    "iso-8859-1",
    schema_estabelecimentos
)

In [44]:
df_estabelecimentos.show(truncate=False)

+---------+----------+-------+-------------+--------------------+------------------+-----------------------+-------------------------+--------------------+----+---------------------+-----------+-----------------------------------------------------------------------+---------------+-------------------------------+------+---------------------------------+-------------------+--------+---+----------------+----+---------+----+---------+-------+--------+---------------------------+-----------------+----------------------+
|cnpj_base|cnpj_ordem|cnpj_dv|matriz_filial|nome_fantasia       |situacao_cadastral|data_situacao_cadastral|motivo_situacao_cadastral|nome_cidade_exterior|pais|data_inicio_atividade|cnae_fiscal|cnae_fiscal_secundaria                                                 |tipo_logradouro|logradouro                     |numero|complemento                      |bairro             |cep     |uf |codigo_municipio|ddd1|telefone1|ddd2|telefone2|ddd_fax|fax     |email                     

In [45]:
df_estabelecimentos = df_estabelecimentos.withColumn(
    "data_ingestao",
    f.current_timestamp()
)

df_estabelecimentos.show(truncate=False)

+---------+----------+-------+-------------+--------------------+------------------+-----------------------+-------------------------+--------------------+----+---------------------+-----------+-----------------------------------------------------------------------+---------------+-------------------------------+------+---------------------------------+-------------------+--------+---+----------------+----+---------+----+---------+-------+--------+---------------------------+-----------------+----------------------+--------------------------+
|cnpj_base|cnpj_ordem|cnpj_dv|matriz_filial|nome_fantasia       |situacao_cadastral|data_situacao_cadastral|motivo_situacao_cadastral|nome_cidade_exterior|pais|data_inicio_atividade|cnae_fiscal|cnae_fiscal_secundaria                                                 |tipo_logradouro|logradouro                     |numero|complemento                      |bairro             |cep     |uf |codigo_municipio|ddd1|telefone1|ddd2|telefone2|ddd_fax|fax     

In [46]:
# Salvar na camada RAW

gravar_delta(df_estabelecimentos, "../RAW/estabelecimentos")

25/07/24 10:11:17 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/07/24 10:11:17 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/07/24 10:11:18 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/07/24 10:11:18 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/07/24 10:11:18 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
25/07/24 10:11:45 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/07/24 10:11:46 WARN MemoryManager: Total allocation exceeds 95.

Gravação efetuada!


DataFrame[cnpj_base: string, cnpj_ordem: string, cnpj_dv: string, matriz_filial: string, nome_fantasia: string, situacao_cadastral: string, data_situacao_cadastral: string, motivo_situacao_cadastral: string, nome_cidade_exterior: string, pais: string, data_inicio_atividade: string, cnae_fiscal: string, cnae_fiscal_secundaria: string, tipo_logradouro: string, logradouro: string, numero: string, complemento: string, bairro: string, cep: string, uf: string, codigo_municipio: string, ddd1: string, telefone1: string, ddd2: string, telefone2: string, ddd_fax: string, fax: string, email: string, situacao_especial: string, data_situacao_especial: string, data_ingestao: timestamp]

In [ ]:
try:
    deltaTable_estabelecimentos = DeltaTable.forPath(spark, "../RAW/estabelecimentos")
    df_estabelecimentos = deltaTable_estabelecimentos.toDF()
    df_estabelecimentos.printSchema()
    df_estabelecimentos.show(5, truncate=False)
except Exception as e:
    print(f"[ERRO] Falha na leiruta do DeltaTable: {e}")
    raise

root
 |-- cnpj_base: string (nullable = true)
 |-- cnpj_ordem: string (nullable = true)
 |-- cnpj_dv: string (nullable = true)
 |-- matriz_filial: string (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- situacao_cadastral: string (nullable = true)
 |-- data_situacao_cadastral: string (nullable = true)
 |-- motivo_situacao_cadastral: string (nullable = true)
 |-- nome_cidade_exterior: string (nullable = true)
 |-- pais: string (nullable = true)
 |-- data_inicio_atividade: string (nullable = true)
 |-- cnae_fiscal: string (nullable = true)
 |-- cnae_fiscal_secundaria: string (nullable = true)
 |-- tipo_logradouro: string (nullable = true)
 |-- logradouro: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- complemento: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- uf: string (nullable = true)
 |-- codigo_municipio: string (nullable = true)
 |-- ddd1: string (nullable = true)
 |-- telefone1: string

## RAW - Socios

In [48]:
# Caminho do csv
path_csv_socios: str = "../LND/socios/2025-06/Socios1_2025-07-23_212245.csv"

In [49]:
# Definição do schema
schema_socios: StructType = StructType([
    StructField("cnpj_basico", StringType(), True),
    StructField("identificador_socio", StringType(), True),
    StructField("nome_socio_razao_social", StringType(), True),
    StructField("cnpj_cpf_socio", StringType(), True),
    StructField("codigo_qualificacao_socio", StringType(), True),
    StructField("data_entrada_sociedade", StringType(), True),
    StructField("pais", StringType(), True),
    StructField("representante_legal", StringType(), True),
    StructField("nome_representante", StringType(), True),
    StructField("codigo_qualificacao_representante", StringType(), True),
    StructField("faixa_etaria", StringType(), True),
])

In [50]:
# Ler csv
df_socios = ler_csv(path=path_csv_socios, encoding="iso-8859-1", schema=schema_socios)

In [51]:
df_socios.show(5, truncate=False)

+-----------+-------------------+----------------------------+--------------+-------------------------+----------------------+----+-------------------+------------------+---------------------------------+------------+
|cnpj_basico|identificador_socio|nome_socio_razao_social     |cnpj_cpf_socio|codigo_qualificacao_socio|data_entrada_sociedade|pais|representante_legal|nome_representante|codigo_qualificacao_representante|faixa_etaria|
+-----------+-------------------+----------------------------+--------------+-------------------------+----------------------+----+-------------------+------------------+---------------------------------+------------+
|07396865   |2                  |GERSON HOFFMANN             |***240659**   |49                       |20050518              |NULL|***000000**        |NULL              |00                               |5           |
|07396865   |2                  |MITZI HARTMANN              |***126339**   |22                       |20080509              |NU

In [52]:
df_socios = df_socios.withColumn(
    "data_ingestao",
    f.current_timestamp()
)

df_socios.show(truncate=False)

+-----------+-------------------+--------------------------------+--------------+-------------------------+----------------------+----+-------------------+------------------+---------------------------------+------------+--------------------------+
|cnpj_basico|identificador_socio|nome_socio_razao_social         |cnpj_cpf_socio|codigo_qualificacao_socio|data_entrada_sociedade|pais|representante_legal|nome_representante|codigo_qualificacao_representante|faixa_etaria|data_ingestao             |
+-----------+-------------------+--------------------------------+--------------+-------------------------+----------------------+----+-------------------+------------------+---------------------------------+------------+--------------------------+
|07396865   |2                  |GERSON HOFFMANN                 |***240659**   |49                       |20050518              |NULL|***000000**        |NULL              |00                               |5           |2025-07-24 10:17:03.169452|
|073

In [53]:
# salvar na camada RAW

gravar_delta(df_socios, "../RAW/socios")

25/07/24 10:17:25 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/07/24 10:17:25 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/07/24 10:17:25 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/07/24 10:17:25 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/07/24 10:17:25 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
25/07/24 10:17:29 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/07/24 10:17:29 WARN MemoryManager: Total allocation exceeds 95.

Gravação efetuada!


DataFrame[cnpj_basico: string, identificador_socio: string, nome_socio_razao_social: string, cnpj_cpf_socio: string, codigo_qualificacao_socio: string, data_entrada_sociedade: string, pais: string, representante_legal: string, nome_representante: string, codigo_qualificacao_representante: string, faixa_etaria: string, data_ingestao: timestamp]

In [55]:
try:
    deltaTable_socios = DeltaTable.forPath(spark, "../RAW/socios")
    df_socios = deltaTable_socios.toDF()
    df_socios.printSchema()
    df_socios.show(5, truncate=False)
except Exception as e:
    print(f"[ERRO] Falha na leitura: {e}")
    raise

root
 |-- cnpj_basico: string (nullable = true)
 |-- identificador_socio: string (nullable = true)
 |-- nome_socio_razao_social: string (nullable = true)
 |-- cnpj_cpf_socio: string (nullable = true)
 |-- codigo_qualificacao_socio: string (nullable = true)
 |-- data_entrada_sociedade: string (nullable = true)
 |-- pais: string (nullable = true)
 |-- representante_legal: string (nullable = true)
 |-- nome_representante: string (nullable = true)
 |-- codigo_qualificacao_representante: string (nullable = true)
 |-- faixa_etaria: string (nullable = true)
 |-- data_ingestao: timestamp (nullable = true)

+-----------+-------------------+----------------------------+--------------+-------------------------+----------------------+----+-------------------+------------------+---------------------------------+------------+--------------------------+
|cnpj_basico|identificador_socio|nome_socio_razao_social     |cnpj_cpf_socio|codigo_qualificacao_socio|data_entrada_sociedade|pais|representante_lega

## RAW - Paises

In [56]:
# Caminho do arquivo
path_csv_paises: str = "../LND/paises/2025-06/Paises_2025-07-23_212003.csv"

In [57]:
# Definição do schema
schema_pais: StructType = StructType([
    StructField("codigo_pais", StringType(), True),
    StructField("nome_pais", StringType(), True),
])

In [58]:
# Ler arquivo CSV
df_paises = ler_csv(path_csv_paises, "iso-8859-1", schema_natureza_juridica)

In [59]:
# Mostrar resultado
df_paises.show(truncate=False)

+------------------------+-----------------------------+
|codigo_natureza_juridica|natureza_juridica            |
+------------------------+-----------------------------+
|000                     |COLIS POSTAUX                |
|013                     |AFEGANISTAO                  |
|017                     |ALBANIA                      |
|020                     |ALBORAN-PEREJIL,ILHAS        |
|023                     |ALEMANHA                     |
|025                     |ALEMANHA, REP. DEMOCRATICA DA|
|031                     |BURKINA FASO                 |
|037                     |ANDORRA                      |
|040                     |ANGOLA                       |
|041                     |ANGUILLA                     |
|043                     |ANTIGUA E BARBUDA            |
|047                     |ANTILHAS HOLANDESAS          |
|053                     |ARABIA SAUDITA               |
|059                     |ARGELIA                      |
|063                     |ARGEN

In [60]:
df_paises = df_paises.withColumn(
    "data_ingestao",
    f.current_timestamp()
)

df_paises.show(truncate=False)

+------------------------+-----------------------------+--------------------------+
|codigo_natureza_juridica|natureza_juridica            |data_ingestao             |
+------------------------+-----------------------------+--------------------------+
|000                     |COLIS POSTAUX                |2025-07-24 10:19:50.758357|
|013                     |AFEGANISTAO                  |2025-07-24 10:19:50.758357|
|017                     |ALBANIA                      |2025-07-24 10:19:50.758357|
|020                     |ALBORAN-PEREJIL,ILHAS        |2025-07-24 10:19:50.758357|
|023                     |ALEMANHA                     |2025-07-24 10:19:50.758357|
|025                     |ALEMANHA, REP. DEMOCRATICA DA|2025-07-24 10:19:50.758357|
|031                     |BURKINA FASO                 |2025-07-24 10:19:50.758357|
|037                     |ANDORRA                      |2025-07-24 10:19:50.758357|
|040                     |ANGOLA                       |2025-07-24 10:19:50.

In [ ]:
# Gravar no RAW
try:
    gravar_delta(df_paises, "../RAW/paises")
except Exception as e:
    print(f"[ERRO] Falha na escrita do DeltaTable")
    raise

Gravação efetuada!


In [62]:
try:
    # Transformar em DeltaTable e mostrar arquivo
    deltaTable_paises = DeltaTable.forPath(spark, "../RAW/paises")
    df_paises = deltaTable_paises.toDF()
    df_paises.printSchema()
    df_paises.show(5, truncate=False)
except Exception as e:
    print(f"[ERRO] Falha na leitura do DeltaTable: {e}")
    raise

root
 |-- codigo_natureza_juridica: string (nullable = true)
 |-- natureza_juridica: string (nullable = true)
 |-- data_ingestao: timestamp (nullable = true)

+------------------------+---------------------+--------------------------+
|codigo_natureza_juridica|natureza_juridica    |data_ingestao             |
+------------------------+---------------------+--------------------------+
|000                     |COLIS POSTAUX        |2025-07-24 10:20:23.372627|
|013                     |AFEGANISTAO          |2025-07-24 10:20:23.372627|
|017                     |ALBANIA              |2025-07-24 10:20:23.372627|
|020                     |ALBORAN-PEREJIL,ILHAS|2025-07-24 10:20:23.372627|
|023                     |ALEMANHA             |2025-07-24 10:20:23.372627|
+------------------------+---------------------+--------------------------+
only showing top 5 rows


## RAW - Qualificações

In [63]:
# Caminho do arquivo
path_csv_qualificacoes: str = "../LND/qualificacoes/2025-06/Qualificacoes_2025-07-23_212004.csv"

In [64]:
# Definição do schema
schema_qualificacoes: StructType = StructType([
    StructField("codigo_qualificacao", StringType(), True),
    StructField("descricao_qualificacao", StringType(), True),
])

In [68]:
# ler arquivo CSV
df_qualificacoes = ler_csv(path_csv_qualificacoes, "iso-8859-1", schema_qualificacoes)
df_qualificacoes.printSchema()
df_qualificacoes.show(5, truncate=False)

root
 |-- codigo_qualificacao: string (nullable = true)
 |-- descricao_qualificacao: string (nullable = true)

+-------------------+----------------------------+
|codigo_qualificacao|descricao_qualificacao      |
+-------------------+----------------------------+
|00                 |Não informada               |
|05                 |Administrador               |
|08                 |Conselheiro de Administração|
|09                 |Curador                     |
|10                 |Diretor                     |
+-------------------+----------------------------+
only showing top 5 rows


In [ ]:
df_qualificacoes = df_qualificacoes.withColumn(
    "data_ingestao",
    f.current_timestamp()
)

df_qualificacoes.show(5, truncate=False)

+-------------------+----------------------------+--------------------------+
|codigo_qualificacao|descricao_qualificacao      |data_ingestao             |
+-------------------+----------------------------+--------------------------+
|00                 |Não informada               |2025-07-24 10:24:24.016977|
|05                 |Administrador               |2025-07-24 10:24:24.016977|
|08                 |Conselheiro de Administração|2025-07-24 10:24:24.016977|
|09                 |Curador                     |2025-07-24 10:24:24.016977|
|10                 |Diretor                     |2025-07-24 10:24:24.016977|
|11                 |Interventor                 |2025-07-24 10:24:24.016977|
|12                 |Inventariante               |2025-07-24 10:24:24.016977|
|13                 |Liquidante                  |2025-07-24 10:24:24.016977|
|14                 |Mãe                         |2025-07-24 10:24:24.016977|
|15                 |Pai                         |2025-07-24 10:

In [70]:
# Salvar no RAW
try:
    gravar_delta(df_qualificacoes, "../RAW/qualificacoes")
except Exception as e:
    print(f"[ERRO] Falha na escrita do DeltaTable")

Gravação efetuada!


In [71]:
try:
    deltaTable_qualificacoes = DeltaTable.forPath(spark, "../RAW/qualificacoes")
    df_qualificacoes = deltaTable_qualificacoes.toDF()
    df_qualificacoes.printSchema()
    df_qualificacoes.show(5, truncate=False)
except Exception as e:
    print(f"Erro: {e}")

root
 |-- codigo_qualificacao: string (nullable = true)
 |-- descricao_qualificacao: string (nullable = true)
 |-- data_ingestao: timestamp (nullable = true)

+-------------------+----------------------------+--------------------------+
|codigo_qualificacao|descricao_qualificacao      |data_ingestao             |
+-------------------+----------------------------+--------------------------+
|00                 |Não informada               |2025-07-24 10:24:45.591468|
|05                 |Administrador               |2025-07-24 10:24:45.591468|
|08                 |Conselheiro de Administração|2025-07-24 10:24:45.591468|
|09                 |Curador                     |2025-07-24 10:24:45.591468|
|10                 |Diretor                     |2025-07-24 10:24:45.591468|
+-------------------+----------------------------+--------------------------+
only showing top 5 rows


## RAW - Motivos

In [72]:
# caminho do CSV
path_csv_motivos: str = "../LND/motivos/2025-06/Motivos_2025-07-23_211903.csv"

In [73]:
# Definição do schema
schema_motivos: StructType = StructType([
    StructField("codigo_motivo", StringType(), True),
    StructField("descricao_motivo", StringType(), True),
]) 

In [75]:
# Ler arquivo CSV
df_motivos: DataFrame = ler_csv(path_csv_motivos, "iso-8859-1", schema_motivos)
df_motivos.printSchema()
df_motivos.show(truncate=False)

root
 |-- codigo_motivo: string (nullable = true)
 |-- descricao_motivo: string (nullable = true)

+-------------+-------------------------------------------------+
|codigo_motivo|descricao_motivo                                 |
+-------------+-------------------------------------------------+
|00           |SEM MOTIVO                                       |
|01           |EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA  |
|02           |INCORPORACAO                                     |
|03           |FUSAO                                            |
|04           |CISAO TOTAL                                      |
|05           |ENCERRAMENTO DA FALENCIA                         |
|06           |ENCERRAMENTO DA LIQUIDACAO                       |
|07           |ELEVACAO A MATRIZ                                |
|08           |TRANSPASSE                                       |
|09           |NAO INICIO DE ATIVIDADE                          |
|10           |EXTINCAO PELO ENCERRAMENTO D

In [76]:
df_motivos = df_motivos.withColumn(
    "data_ingestao",
    f.current_timestamp()
)

df_motivos.show(truncate=False)

+-------------+-------------------------------------------------+--------------------------+
|codigo_motivo|descricao_motivo                                 |data_ingestao             |
+-------------+-------------------------------------------------+--------------------------+
|00           |SEM MOTIVO                                       |2025-07-24 10:27:09.543976|
|01           |EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA  |2025-07-24 10:27:09.543976|
|02           |INCORPORACAO                                     |2025-07-24 10:27:09.543976|
|03           |FUSAO                                            |2025-07-24 10:27:09.543976|
|04           |CISAO TOTAL                                      |2025-07-24 10:27:09.543976|
|05           |ENCERRAMENTO DA FALENCIA                         |2025-07-24 10:27:09.543976|
|06           |ENCERRAMENTO DA LIQUIDACAO                       |2025-07-24 10:27:09.543976|
|07           |ELEVACAO A MATRIZ                                |2025-

In [77]:
# Gravar no RAW
try:
    gravar_delta(df_motivos, "../RAW/motivos")
except Exception as e:
    print(f"[ERRO] Falha ao gravar DeltaTable")
    raise

Gravação efetuada!


In [78]:
try:
    deltaTable_motivos: DeltaTable = DeltaTable.forPath(spark, "../RAW/motivos")
    df_motivos: DataFrame = deltaTable_motivos.toDF()
    df_motivos.printSchema()
    df_motivos.show(5, truncate=False)
except Exception as e:
    print(f"[ERRO] Falha na leitura do DeltaTable: {e}")

root
 |-- codigo_motivo: string (nullable = true)
 |-- descricao_motivo: string (nullable = true)
 |-- data_ingestao: timestamp (nullable = true)

+-------------+-----------------------------------------------+--------------------------+
|codigo_motivo|descricao_motivo                               |data_ingestao             |
+-------------+-----------------------------------------------+--------------------------+
|00           |SEM MOTIVO                                     |2025-07-24 10:27:48.010526|
|01           |EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA|2025-07-24 10:27:48.010526|
|02           |INCORPORACAO                                   |2025-07-24 10:27:48.010526|
|03           |FUSAO                                          |2025-07-24 10:27:48.010526|
|04           |CISAO TOTAL                                    |2025-07-24 10:27:48.010526|
+-------------+-----------------------------------------------+--------------------------+
only showing top 5 rows


## RAW - Municipios Receita Federal

In [79]:
# caminho do CSV
path_csv_municipios_rf: str = "../LND/municipios/2025-06/Municipios_2025-07-23_211903.csv"

In [80]:
schema_municipios_rf: StructType = StructType([
    StructField("id_municipio", StringType(), True),
    StructField("municipio", StringType(), True)
])

In [81]:
# Ler arquivo CSV
df_municipios_rf: DataFrame = ler_csv(path_csv_municipios_rf, "iso-8859-1", schema_municipios_rf)
df_municipios_rf.printSchema()
df_municipios_rf.show(5, truncate=False)

root
 |-- id_municipio: string (nullable = true)
 |-- municipio: string (nullable = true)

+------------+-----------------------+
|id_municipio|municipio              |
+------------+-----------------------+
|0001        |GUAJARA-MIRIM          |
|0002        |ALTO ALEGRE DOS PARECIS|
|0003        |PORTO VELHO            |
|0004        |BURITIS                |
|0005        |JI-PARANA              |
+------------+-----------------------+
only showing top 5 rows


In [82]:
df_municipios_rf = df_municipios_rf.withColumn(
    "data_ingestao",
    f.current_timestamp()
)

df_municipios_rf.show(truncate=False)

+------------+------------------------+--------------------------+
|id_municipio|municipio               |data_ingestao             |
+------------+------------------------+--------------------------+
|0001        |GUAJARA-MIRIM           |2025-07-24 10:30:10.300014|
|0002        |ALTO ALEGRE DOS PARECIS |2025-07-24 10:30:10.300014|
|0003        |PORTO VELHO             |2025-07-24 10:30:10.300014|
|0004        |BURITIS                 |2025-07-24 10:30:10.300014|
|0005        |JI-PARANA               |2025-07-24 10:30:10.300014|
|0006        |CHUPINGUAIA             |2025-07-24 10:30:10.300014|
|0007        |ARIQUEMES               |2025-07-24 10:30:10.300014|
|0008        |CUJUBIM                 |2025-07-24 10:30:10.300014|
|0009        |CACOAL                  |2025-07-24 10:30:10.300014|
|0010        |NOVA UNIAO              |2025-07-24 10:30:10.300014|
|0011        |PIMENTA BUENO           |2025-07-24 10:30:10.300014|
|0012        |PARECIS                 |2025-07-24 10:30:10.300

In [83]:
# Gravar no RAW

try:
    gravar_delta(df_municipios_rf, "../RAW/municipios_rf")
except Exception as e:
    print(f"[ERRO] Falha ao gravar DeltaTable")
    raise

Gravação efetuada!


In [84]:
try:
    deltaTable_municipios_rf: DeltaTable = DeltaTable.forPath(spark, "../RAW/municipios_rf")
    df_municipios_rf: DataFrame = deltaTable_municipios_rf.toDF()
    df_municipios_rf.printSchema()
    df_municipios_rf.show(5, truncate=False)
except Exception as e:
    print(f"[ERRO] Falha na leitura do DeltaTable: {e}")

root
 |-- id_municipio: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- data_ingestao: timestamp (nullable = true)

+------------+-----------------------+--------------------------+
|id_municipio|municipio              |data_ingestao             |
+------------+-----------------------+--------------------------+
|0001        |GUAJARA-MIRIM          |2025-07-24 10:31:21.883376|
|0002        |ALTO ALEGRE DOS PARECIS|2025-07-24 10:31:21.883376|
|0003        |PORTO VELHO            |2025-07-24 10:31:21.883376|
|0004        |BURITIS                |2025-07-24 10:31:21.883376|
|0005        |JI-PARANA              |2025-07-24 10:31:21.883376|
+------------+-----------------------+--------------------------+
only showing top 5 rows


## Finalizar spark

In [85]:
spark.stop()